## Sztuczna inteligencja i inżynieria wiedzy 
### Lista 4
##### Aleksander Stepaniuk 272644



**Kroki realizacji:**  
1. Eksploracja danych  
2. Przygotowanie danych  
3. Trenowanie i strojenie modeli (testowanie klasyfikatorów)
4. Ocena wyników klasyfikacji i wnioski

**Napotkane trudności:**  
- Imbalans klas (najmniej próbek w klasach 3 i 5)
- Wysoka wariancja niektórych cech
- Konieczność strojenia hiperparametrów dla różnych algorytmów


# ZAD 1. Eksploracja danych

In [2]:
import pandas as pd
from ucimlrepo import fetch_ucirepo

# fetch datasetu
cardiotocography = fetch_ucirepo(id=193)
X = cardiotocography.data.features
y = cardiotocography.data.targets

# 1.1 podgląd pierwszych wierszy
print(X.head())

# 1.2 podstawowe statystyki opisowe
print(X.describe())

# 1.3 rozkład klas
print(y['CLASS'].value_counts().sort_index())

# 1.4 liczba brakujących wartości w każdej kolumnie
print(X.isna().sum())

    LB     AC   FM     UC     DL   DS   DP  ASTV  MSTV  ALTV  ...  Width  Min  \
0  120  0.000  0.0  0.000  0.000  0.0  0.0    73   0.5    43  ...     64   62   
1  132  0.006  0.0  0.006  0.003  0.0  0.0    17   2.1     0  ...    130   68   
2  133  0.003  0.0  0.008  0.003  0.0  0.0    16   2.1     0  ...    130   68   
3  134  0.003  0.0  0.008  0.003  0.0  0.0    16   2.4     0  ...    117   53   
4  132  0.007  0.0  0.008  0.000  0.0  0.0    16   2.4     0  ...    117   53   

   Max  Nmax  Nzeros  Mode  Mean  Median  Variance  Tendency  
0  126     2       0   120   137     121        73         1  
1  198     6       1   141   136     140        12         0  
2  198     5       1   141   135     138        13         0  
3  170    11       0   137   134     137        13         1  
4  170     9       0   137   136     138        11         1  

[5 rows x 21 columns]
                LB           AC           FM           UC           DL  \
count  2126.000000  2126.000000  2126.

## Uwagi po eksploracji danych (ZAD 1):

- Brak braków danych – wszystkie kolumny mają 2126 wartości (w poleceniu jest o brakującym zbiorze, jednak nie został on dostarczony, więc pracujemy nad oryginalnym zbiorem z: https://archive.ics.uci.edu/dataset/193/cardiotocography
- Rozkład LB (bazowe FHR czyli fetal heart rate): średnio ~133 bpm, od 106 do 160, odchylenie standardowe w przybliżeniu: 9,8
- Zmienność krótkoterminowa (krótkotrwałe wahanie tętna) (ASTV, MSTV) i długoterminowa (ALTV, MLTV) krzywo rozłożone – duży wpływ kilku ekstremów
- Histogram FHR: szeroka rozpiętość wariancji (do 269), tendencja przyjmuje wartości –1, 0, 1 (kategoria)
- Brak balansu klas: najwięcej próbek w klasie 2 (579) i 1 (384), najmniej w 3 (53) i 5 (72)

# ZAD 2. Przygotowanie danych

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import PCA

# 2.1 podział na zestaw uczący i testowy
X_train, X_test, y_train, y_test = train_test_split(
    X, y['CLASS'],
    test_size=0.3,
    stratify=y['CLASS'],
    random_state=42
)

# 2.2 imputacja braków (mean) – tu zbiór już teoretycznie bez brakujących wartości
imputer = SimpleImputer(strategy='mean')
X_train_imp = imputer.fit_transform(X_train)
X_test_imp = imputer.transform(X_test)

# 2.3 różne sposoby przetwarzania:

# a) standaryzacja
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train_imp)
X_test_std = scaler.transform(X_test_imp)

# b) normalizacja do długości wektora = 1
normalizer = Normalizer()
X_train_norm = normalizer.fit_transform(X_train_imp)
X_test_norm = normalizer.transform(X_test_imp)

# c) PCA zachowujące 95% wariancji (na danych wystandaryzowanych)
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

# 2.4 kształty macierzy
print("raw imp:", X_train_imp.shape, X_test_imp.shape)
print("std:",     X_train_std.shape,  X_test_std.shape)
print("norm:",    X_train_norm.shape, X_test_norm.shape)
print("pca:",     X_train_pca.shape,  X_test_pca.shape)


raw imp: (1488, 21) (638, 21)
std: (1488, 21) (638, 21)
norm: (1488, 21) (638, 21)
pca: (1488, 14) (638, 14)


## Uwagi po przygotowaniu danych (ZAD 2):
- Został utworzony podział na zbiory uczący i testowy z zachowaniem proporcji klas
- Imputacja braków nie była konieczna przez kompletny zbiór danych, ale została wykonana dla zasady (i pokazania możliwości)
- Standaryzacja i normalizacja zmieniają rozkład cech, co może poprawić wyniki modeli

# ZAD 3. Modelowanie

In [4]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# helper: train & eval
def train_eval(model, X_tr, y_tr, X_te, y_te):
    model.fit(X_tr, y_tr) # na treningowych się uczy
    y_pred = model.predict(X_te) # predykcje robi na testowych
    print(f"{model.__class__.__name__}: accuracy =",
          round(accuracy_score(y_te, y_pred), 3))

# 3.1 Gaussian Naive Bayes (naiwny klasyfikator Bayesa)
gnb = GaussianNB()
train_eval(gnb, X_train_std, y_train, X_test_std, y_test)

# 3.2 Decision Tree – 3 zestawy hiperparametrów
dt_params = [
    {"random_state":42}, # stały seed dla powtarzalności wyników
    {"max_depth":5, "random_state":42}, # ograniczenie głębokości drzewa żeby uniknąć przeuczenia (overfitting)
    {"min_samples_split":10, "random_state":42} # wymaga co najmniej 10 próbek aby ograniczyć wzrost drzewa (overfitting)
]
for params in dt_params:
    dt = DecisionTreeClassifier(**params)
    train_eval(dt, X_train_std, y_train, X_test_std, y_test)

# 3.3 Random Forest (bonusowy)
rf_params = [
    {"n_estimators":100, "random_state":42}, # liczba drzew w lesie, 100 to standardowa wartość
    {"n_estimators":200, "max_depth":7, "random_state":42} # dodatkowe ograniczenie głębokości każdego drzewa
]
for params in rf_params:
    rf = RandomForestClassifier(**params)
    train_eval(rf, X_train_std, y_train, X_test_std, y_test)

# 3.4 SVM - Support Vector Machines (bonusowy)
svm_models = [
    SVC(kernel="linear", random_state=42),
    SVC(kernel="rbf", C=1.0, random_state=42)
]
for svm in svm_models:
    train_eval(svm, X_train_std, y_train, X_test_std, y_test)

# 3.5 łagodzenie przeuczenia dla drzewa: cost-complexity pruning, porównanie default vs ccp_alpha=0.01
# ccp_alpha = 0 -> brak przycinania, większe ryzyko overfittingu
# ccp_alpha = 0.01 -> przycinanie gałęzie drzewa które nie poprawiają znacząco jakości podziału aby uprościć drzewo
dt_default = DecisionTreeClassifier(random_state=42)
dt_pruned  = DecisionTreeClassifier(ccp_alpha=0.01, random_state=42)
print("\npruning:")
train_eval(dt_default, X_train_std, y_train, X_test_std, y_test)
train_eval(dt_pruned,  X_train_std, y_train, X_test_std, y_test)


GaussianNB: accuracy = 0.586
DecisionTreeClassifier: accuracy = 0.82
DecisionTreeClassifier: accuracy = 0.779
DecisionTreeClassifier: accuracy = 0.809
RandomForestClassifier: accuracy = 0.862
RandomForestClassifier: accuracy = 0.817
SVC: accuracy = 0.807
SVC: accuracy = 0.785

pruning:
DecisionTreeClassifier: accuracy = 0.82
DecisionTreeClassifier: accuracy = 0.765


## Uwagi po modelowaniu (ZAD 3):

| Model                                           | Accuracy |
|-------------------------------------------------|----------|
| GaussianNB                                      | 0.586    |
| DecisionTree (default)                          | 0.820    |
| DecisionTree (max\_depth=5)                     | 0.779    |
| DecisionTree (min\_samples\_split=10)           | 0.809    |
| RandomForest (n\_estimators=100)                | 0.862    |
| RandomForest (n\_estimators=200, max\_depth=7)  | 0.817    |
| SVM (kernel=linear)                             | 0.807    |
| SVM (kernel=rbf, C=1.0)                         | 0.785    |
| DecisionTree (default)                          | 0.820    |
| DecisionTree (ccp\_alpha=0.01, pruned)          | 0.765    |

**Wyniki:**  
- GaussianNB ma najniższą dokładność (0.586), co sugeruje, że dane nie są dobrze rozdzielone liniowo i wymagają bardziej złożonych modeli (nie pokrywa się z danymi CTG)
- DecisionTree bez ograniczeń osiąga 0.820, ale łatwo przeucza się na szumie
- DecisionTree z ograniczeniem głębokości (max_depth=5) spada do 0.779, więc ogranicenie głębokości obniża dokładność, drzewo jest zbyt płytkie (underfitting)
- DecisionTree z ograniczeniem min_samples_split=10 osiąga 0.809, co jest kompromisem między złożonością a przeuczeniem
- Random Forest (100 drzew) to zdecydowanie najlepszy model, osiągający najwyższą dokładność (accuracy 0.862)
- Random Forest z 200 drzewami i ograniczeniem głębokości (max_depth=7) osiąga 0.817, co jest nieco gorsze niż 100 drzew bez ograniczeń, może zbyt restrykcyjne ograniczenie głębokości

**Podsumowanie:**
- GaussianNB nie radzi sobie z nieregularnymi podziałami danych, co jest typowe dla danych CTG
- Drzewa pojedyncze potrzebują starannego przycinania albo innych ograniczeń, by unikać under/overfittingu
- SVM i Bayes dobrze pokazują, gdzie konieczne jest dalsze strojenie lub zmiana założeń
- Random Forest jest najbardziej stabilnym i skutecznym modelem, który dobrze radzi sobie z nieregularnymi podziałami danych



# ZAD 4. Wnioski

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_std, y_train)
y_pred = rf.predict(X_test_std)

print("Precision (macro):", round(precision_score(y_test, y_pred, average='macro'), 3))
print("Recall    (macro):", round(recall_score   (y_test, y_pred, average='macro'), 3))
print("F1-score  (macro):", round(f1_score       (y_test, y_pred, average='macro'), 3))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


Precision (macro): 0.858
Recall    (macro): 0.796
F1-score  (macro): 0.817
Confusion matrix:
 [[ 98   5   1   0   1   1   4   0   0   5]
 [  6 161   1   2   2   2   0   0   0   0]
 [  3   2  10   0   0   0   1   0   0   0]
 [  0   8   0  15   0   1   0   0   0   0]
 [  6   3   0   0   8   0   0   0   0   4]
 [  0  10   0   0   0  89   1   0   0   0]
 [  1   0   0   0   0   6  65   3   0   1]
 [  0   0   0   0   0   0   2  30   0   0]
 [  1   0   0   0   0   0   0   0  20   0]
 [  3   0   0   0   0   0   0   0   2  54]]


## Interpretacja metryk i macierzy pomyłek (RandomForest n=100)

- **Precision (macro) = 0.858**  
  Średnio dla wszystkich klas model popełnia raczej niezbyt wiele fałszywych pozytywów (gdy twierdzi, że należy do klasy X, zwykle ma racje)

- **Recall (macro) = 0.796**  
  Średnio model znajduje prawie 80% wszystkich prawdziwych próbek każdej klasy, niektóre klasy (te rzadsze) mają niższy recall, bo brak danych utrudnia ich wyłapywanie

- **F1-score (macro) = 0.817**  
  Uśredniona miara łącząca precision i recall – pokazuje dobrą równowagę między nimi.

- **Macierz pomyłek**

    -   |        | 1  | 2   | 3  | 4  | 5 | 6  | 7  | 8  | 9  | 10 |
        |--------|----|-----|----|----|---|----|----|----|----|----|
        | **1**  | 98 | 5   | 1  | 0  | 1 | 1  | 4  | 0  | 0  | 5  |
        | **2**  | 6  | 161 | 1  | 2  | 2 | 2  | 0  | 0  | 0  | 0  |
        | **3**  | 3  | 2   | 10 | 0  | 0 | 0  | 1  | 0  | 0  | 0  |
        | **4**  | 0  | 8   | 0  | 15 | 0 | 1  | 0  | 0  | 0  | 0  |
        | **5**  | 6  | 3   | 0  | 0  | 8 | 0  | 0  | 0  | 0  | 4  |
        | **6**  | 0  | 10  | 0  | 0  | 0 | 89 | 1  | 0  | 0  | 0  |
        | **7**  | 1  | 0   | 0  | 0  | 0 | 6  | 65 | 3  | 0  | 1  |
        | **8**  | 0  | 0   | 0  | 0  | 0 | 0  | 2  | 30 | 0  | 0  |
        | **9**  | 1  | 0   | 0  | 0  | 0 | 0  | 0  | 0  | 20 | 0  |
        | **10** | 3  | 0   | 0  | 0  | 0 | 0  | 0  | 0  | 2  | 54 |

  - Klasy 1 i 2 (najwięcej próbek) są rozpoznawane bardzo dobrze: mało pomyłek poza swoimi klasami, wysoki recall i precision
  - Klasy 3 i 4: tylko po kilkanaście poprawnych detekcji, sporo pomyłek na sąsiednie klasy -> potrzeba więcej danych lub jakiegoś rodzaju oversamplingu
  - Klasa 5: sporo pomyłek na klasy 1 i 2, ale też kilka poprawnych detekcji, co sugeruje, że model częściowo rozumie tę klasę
  - Klasy 6–10: umiarkowane wyniki, ale klasy 8–10 są już rozpoznawane w większości poprawnie (>70–80%).  

**Wnioski:**  
- Model świetny dla dominujących klas, ale słabszy dla tych z małą liczbą próbek
- Wymaga dalszego strojenia i być może oversamplingu dla klas 3 i 5, które mają najmniej próbek, ewentualnie cost-sensitive learning lub priorytetyzacja recall dla krytycznych wzorców
